<a href="https://colab.research.google.com/github/dmitrirepnikov/code_snippets/blob/master/expansions_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [34]:
from scipy.spatial import distance
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
spreadsheet_id = '13igrV2qBzLDFgo2eOBqfSwaHjAKSZCuU5zqxtbOHPKA'
sheet_id = '0'
url = f'https://docs.google.com/spreadsheets/d/{spreadsheet_id}/export?format=csv&gid={sheet_id}'

df = pd.read_csv(url)

In [36]:
df.head()

,city,coordinates,delivers_per_day,regulatory_score,latitude,longitude
0,Atlanta,"33.19,-84.28",0.00,90,33.19,-84.28
1,Atlanta,"33.2,-84.28",0.01,90,33.20,-84.28
2,Atlanta,"33.2,-84.29",0.01,90,33.20,-84.29
3,Atlanta,"33.21,-84.06",0.00,90,33.21,-84.06
4,Atlanta,"33.23,-84.23",0.01,90,33.23,-84.23


In [47]:
from scipy.stats import gaussian_kde
from sklearn.cluster import DBSCAN
import folium
from folium.plugins import HeatMap

print("DataFrame Info:")
print(df.info())
print("\nFirst few rows of the DataFrame:")
print(df.head())

# Weather scores for each city from chatgpt and also informed from our sheet of avg snow / rain days
weather_scores = {
    'Miami': 0.80, 'New Jersey': 0.6, 'Atlanta': 0.75, 'Houston': 0.80,
    'San Diego': 0.95, 'Orlando': 0.80, 'Tampa Bay': 0.80, 'Phoenix': 0.95
}

## these are the params -

def identify_hotspots(city_data, threshold_percentile=50, min_points=5, max_distance_km=2): # 1 km is roughly the 2 decimals points coords we have
    """Identify hotspots using Kernel Density Estimation and DBSCAN clustering"""
    coords = city_data[['lat', 'lon']].values
    weights = city_data['weather_adjusted_delivers'].values

    kde = gaussian_kde(coords.T, weights=weights)
    density = kde(coords.T)

    threshold = np.percentile(density, threshold_percentile)
    high_density_points = city_data[density > threshold].copy()

    if len(high_density_points) < min_points:
        return city_data, []

    # Cluster high-density points
    kms_per_radian = 6371.0088
    epsilon = max_distance_km / kms_per_radian
    db = DBSCAN(eps=epsilon, min_samples=min_points, metric='haversine').fit(np.radians(high_density_points[['lat', 'lon']]))

    high_density_points['cluster'] = db.labels_

    hotspots = []
    for cluster in set(db.labels_):
        if cluster != -1:  # -1 is noise in DBSCAN
            cluster_points = high_density_points[high_density_points['cluster'] == cluster]
            hotspots.append({
                'center_lat': cluster_points['lat'].mean(),
                'center_lon': cluster_points['lon'].mean(),
                'total_deliveries': cluster_points['weather_adjusted_delivers'].sum(),
                'num_points': len(cluster_points)
            })

    return city_data, hotspots

# splot coords
df[['lat', 'lon']] = df['coordinates'].str.split(',', expand=True).astype(float)

df['weather_adjusted_delivers'] = df.apply(lambda row: row['delivers_per_day'] * weather_scores.get(row['city'], 1), axis=1)

city_results = []
for city in df['city'].unique():
    city_data = df[df['city'] == city].copy()
    try:
        _, hotspots = identify_hotspots(city_data)

        total_deliveries = city_data['weather_adjusted_delivers'].sum()
        hotspot_deliveries = sum(spot['total_deliveries'] for spot in hotspots)
        hotspot_percentage = (hotspot_deliveries / total_deliveries) * 100 if total_deliveries > 0 else 0

        city_results.append({
            'city': city,
            'total_deliveries': total_deliveries,
            'hotspot_deliveries': hotspot_deliveries,
            'hotspot_percentage': hotspot_percentage,
            'num_hotspots': len(hotspots),
            'regulatory_score': city_data['regulatory_score'].iloc[0],
            'weather_score': weather_scores.get(city, 1)
        })

        # Create a map centered on the city
        m = folium.Map(location=[city_data['lat'].mean(), city_data['lon'].mean()], zoom_start=12)

        # Add a heatmap layer
        HeatMap(data=city_data[['lat', 'lon', 'weather_adjusted_delivers']].values.tolist(),
                radius=15, max_zoom=1).add_to(m)

        # Add markers for hotspots
        for spot in hotspots:
            folium.CircleMarker(
                [spot['center_lat'], spot['center_lon']],
                radius=10,
                popup=f"Deliveries: {spot['total_deliveries']:.2f}, Points: {spot['num_points']}",
                color='red',
                fill=True,
                fillColor='red'
            ).add_to(m)

        # Save the map
        m.save(f'{city}_hotspots_map.html')

        print(f"Processed {city}: {len(hotspots)} hotspots, {hotspot_percentage:.2f}% deliveries in hotspots")

    except Exception as e:
        print(f"Error processing {city}: {str(e)}")

# Create a DataFrame with city results
city_scores = pd.DataFrame(city_results)

# Normalize city-level metrics
scaler = MinMaxScaler()
for col in ['total_deliveries', 'hotspot_percentage', 'regulatory_score', 'weather_score']:
    city_scores[f'normalized_{col}'] = scaler.fit_transform(city_scores[[col]])

# Calculate final city score with updated weights
city_scores['city_score'] = (
    0.35 * city_scores['normalized_total_deliveries'] +
    0.35 * city_scores['normalized_hotspot_percentage'] +
    0.3 * city_scores['normalized_regulatory_score']
)

# Rank cities
city_scores['rank'] = city_scores['city_score'].rank(ascending=False, method='dense')

# Sort cities by score
ranked_cities = city_scores.sort_values('city_score', ascending=False)

# Display top cities
print("\nTop Cities Ranked by Attractiveness:")
print(ranked_cities[['city', 'city_score', 'rank', 'total_deliveries', 'hotspot_percentage', 'num_hotspots', 'regulatory_score', 'weather_score']])

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11978 entries, 0 to 11977
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   city                       11978 non-null  object 
 1   coordinates                11978 non-null  object 
 2   delivers_per_day           11978 non-null  float64
 3   regulatory_score           11978 non-null  int64  
 4   latitude                   11978 non-null  float64
 5   longitude                  11978 non-null  float64
 6   lat                        11978 non-null  float64
 7   lon                        11978 non-null  float64
 8   weather_adjusted_delivers  11978 non-null  float64
dtypes: float64(6), int64(1), object(2)
memory usage: 842.3+ KB
None

First few rows of the DataFrame:
      city   coordinates  delivers_per_day  regulatory_score  latitude  \
0  Atlanta  33.19,-84.28              0.00                90     33.19   
1  Atlant

In [45]:
print(ranked_cities[['city', 'city_score', 'rank', 'total_deliveries', 'hotspot_percentage', 'num_hotspots', 'regulatory_score', 'weather_score']].to_csv(sep='\t', index=False))

city	city_score	rank	total_deliveries	hotspot_percentage	num_hotspots	regulatory_score	weather_score
Miami	0.6320327190058682	1.0	3134.5280000000002	77.16683341160136	7	80	0.8
New Jersey	0.5258327948054782	2.0	2069.1839999999997	91.88839658532059	25	70	0.6
Orlando	0.43817889307649244	3.0	644.336	71.35283454595118	6	95	0.8
Atlanta	0.4324271265640639	4.0	922.7774999999999	73.48521176556645	22	90	0.75
San Diego	0.41789109509005706	5.0	603.554	70.03872064471447	6	95	0.95
Tampa Bay	0.39730249217402547	6.0	637.36	73.69900840968998	4	90	0.8
Houston	0.3685295380533037	7.0	945.008	73.46562145505646	11	85	0.8
Phoenix	0.35285988457408096	8.0	754.4994999999999	62.562798252351406	2	95	0.95



In [59]:
## next one for map